In [8]:
import cx_Oracle
import os
from numpy import dot
from numpy.linalg import norm
import numpy as np

In [9]:
def cos_sim(a, b): #코사인 유사도를 뽑는 함수
    return dot(a, b)/(norm(a)*norm(b))  

In [10]:
def errorCheck(userId) :
    LOCATION = "C:\instantclient_21_6"
    os.environ["PATH"] = LOCATION 
    connect = cx_Oracle.connect("campus_f_0707_2","smhrd2","project-db-stu.ddns.net:1524/xe")    ## DB접속
    cursor1 = connect.cursor()
    cursor1.execute(f"select indexu from userui where userid='{userId}'")
    col1=[]
    for i in cursor1:
        col1.append(i)
    cursor1.close()
    if col1 == [] :
        error = 0
    else :
        error = 1
    return error

In [11]:
errorCheck('zxc')

1

## 메뉴 추천
사용자가 전날 섭취했던 음식데이터를 기반으로 기준데이터와 비교해 추천을 한다

In [29]:
def recommendFood(userId,category) : # 추천하는 음식에 이름 리스트를 뽑아주는 함수
    
    LOCATION = "C:\instantclient_21_6"
    os.environ["PATH"] = LOCATION 
    connect = cx_Oracle.connect("campus_f_0707_2","smhrd2","project-db-stu.ddns.net:1524/xe")    ## DB접속
    cursor1 = connect.cursor()
    cursor1.execute(f"select * from food where label='{category}'")  # 분류가 category 인음식 데이터
    cursor2 = connect.cursor()
    cursor2.execute(f"select r.* from reference r, userinfo i where r.gender=i.gender and r.age=i.age and i.userid='{userId}'") # 아이디가 userId인 사람의 성별과 나이대를 고려한 기준데이터
    cursor3 = connect.cursor()
    cursor3.execute(f"select TO_DATE(u.udate, 'YYYY-MM-DD'), avg(f.kcal),avg(f.carbohydrate),avg(f.protein),avg(f.fat),avg(f.sugar),avg(f.ca),avg(f.fe),avg(f.mg),avg(f.k),avg(f.na),avg(f.vitaminb),avg(f.vitaminc)from food f, userui u where f.foodid=u.foodid and u.userid='{userId}' group by TO_DATE(u.udate, 'YYYY-MM-DD') order by TO_DATE(u.udate, 'YYYY-MM-DD') desc") 
    # 아이디가 a인 사람인 섭취한 음식 일평균 영양 데이터
    col1=[]
    col2=[]
    col3=[]
    for i in cursor1:
        col1.append(i)
    for i in cursor2:
        col2.append(i)
    for i in cursor3:
        col3.append(i)
    cursor1.close()
    cursor2.close()
    cursor3.close()   ## 서버 닫기
    reference=np.array(col2[0][3:])  # 기준데이터
    userInfo=np.array(col3[0][1:]) # 최근 일평균 영양데이터 
    percent =1- userInfo/reference #부족한 권장 섭취량과 실제 섭취량의 비율 
    print(len(col1[1]))
    recommend={}
    for i in range(len(col1)) :
        recommend[col1[i][1]]=cos_sim(np.array(col1[i][3:15]),percent) ## foodid 별 category 음식데이터와 비율의 코사인 유사도 구하기
    recommend_list = sorted(recommend.items(), key = lambda item: item[1], reverse = True)  ## 유사도 높은 순으로 정렬
    menuRecommend =[]
    for i in range(5):
        menuRecommend.append(recommend_list[i][0])
    return menuRecommend

In [30]:
recommendFood('zxc','밥류')

16


['곤드레나물밥', '영양돌솥밥', '검정콩밥', '닭고기덮밥', '생선모듬초밥']

## 영양제 추천
사용자가 섭취했던 전체 데이터를 총합 평균내어 기준데이터에 기반에 추천한다

In [ ]:
def deficientNutrient(userId) : # 추천하는 영양제의 성분을 뽑아주는 함수
    LOCATION = "C:\instantclient_21_6"
    os.environ["PATH"] = LOCATION
    connect = cx_Oracle.connect("campus_f_0707_2","smhrd2","project-db-stu.ddns.net:1524/xe")    ## DB접속
 
    cursor2 = connect.cursor()
    cursor2.execute(f"select r.ca,r.fe,r.mg,r.k,r.vitaminb,r.vitaminc from reference r, userinfo i where r.gender=i.gender and r.age=i.age and i.userid='{userId}'") # 아이디가 a인 사람의 성별과 나이대를 고려한 기준데이터
    cursor3 = connect.cursor()
    cursor3.execute(f"select avg(f.ca),avg(f.fe),avg(f.mg),avg(f.k),avg(f.vitaminb),avg(f.vitaminc)from food f, userui u where f.foodid=u.foodid and u.userid='{userId}'") 
    # 아이디가 a인 사람인 섭취한 평균 영양 데이터
    
    col2=[]
    col3=[]
   
    for i in cursor2:
        col2.append(i)
    for i in cursor3:
        col3.append(i)

    reference=np.array(col2[0])  # 기준데이터
    userInfo=np.array(col3[0])
    percent = userInfo/reference #권장 섭취량과 실제 섭취량에 비율
        
    min_list =[]
    for i in range(len(percent)):
        min_list.append(np.argmax(percent))  #가장 큰 인덱스를 반환
        percent[np.argmax(percent)]=-1       #가장 비율이 큰 영양소의 인덱스 순으로 뽑아 리스트에 저장한다
    min_list.reverse()   # 역을 취해 오름차순으로
    lack=0
    if min_list[0]==0 :
        print("ca")
        lack ="ca"   ##
    elif min_list[0]==1:
        print("fe")
        lack ="fe" ##
    elif min_list[0]==2:
        print("mg")
        lack ="mg" ##
    elif min_list[0]==3:
        print("k")
        lack ="k"  ##
    elif min_list[0]==4:
        print("vitaminb")
        lack ="vitaminb" ##
    elif min_list[0]==5:
        print("vitaminc")
        lack ="vitaminc"
   
    cursor2.close()
    cursor3.close()   ## 서버 닫기
  
    return lack

In [17]:
deficientNutrient('zxc')

vitaminb


'vitaminb'

In [18]:
def recommendNutrients(nutrient) : 
    nutrient =nutrient.lower()
    LOCATION = "C:\instantclient_21_6"
    os.environ["PATH"] = LOCATION
    connect = cx_Oracle.connect("campus_f_0707_2","smhrd2","project-db-stu.ddns.net:1524/xe")    ## DB접속

    nutrients0 = connect.cursor()
    nutrients0.execute(f"select nutnm, img, link from nutrients where label='{nutrient}'order by score desc") # 부족한 영양소의 해당하는 영양제 리스트

    col=[]
    nutrients=[]
    for i in nutrients0:
        col.append(i)
    for  i in range(3) :
        nutrients.append(col[i][0])
    return nutrients

In [19]:
recommendNutrients('VITAMINB')

['NOW Foods, 비타민B-6 함유 GABA, 500mg, 베지 캡슐 200정',
 'Solgar, 비타민B6 함유 마그네슘, 250정',
 'Sundown Naturals, B-복합체, 100 정']

### 영양제 추천메커니즘 추가

In [20]:
def recommendUsaUser(userId) :  ## 사용자와 유사한 영양데이터를 갖는 사용자를 찾는 함수
    LOCATION = "C:\instantclient_21_6"
    os.environ["PATH"] = LOCATION
    connect = cx_Oracle.connect("campus_f_0707_2","smhrd2","project-db-stu.ddns.net:1524/xe")    ## DB접속
    cursor1 = connect.cursor() #전체 유저 영양 평균 데이터
    cursor1.execute(f" select userid, avg(f.kcal),avg(f.carbohydrate),avg(f.protein),avg(f.fat),avg(f.sugar),avg(f.ca),avg(f.fe),avg(f.mg),avg(f.k),avg(f.na),avg(f.vitaminb),avg(f.vitaminc)from food f, userui u where f.foodid=u.foodid group by u.userid")
    cursor2 = connect.cursor()
    cursor2.execute(f"select r.* from reference r, userinfo i where r.gender=i.gender and r.age=i.age and i.userid='{userId}'") # 아이디가 userId인 사람의 성별과 나이대를 고려한 기준데이터
    cursor3 = connect.cursor()
    cursor3.execute(f"select avg(f.kcal),avg(f.carbohydrate),avg(f.protein),avg(f.fat),avg(f.sugar),avg(f.ca),avg(f.fe),avg(f.mg),avg(f.k),avg(f.na),avg(f.vitaminb),avg(f.vitaminc)from food f, userui u where f.foodid=u.foodid and u.userid='{userId}'") 
    # 아이디가 a인 사람인 섭취한 음식 영양 데이터
    col1=[]
    col2=[]
    col3=[]
    for i in cursor1:
        col1.append(i)   
    for i in cursor2:
        col2.append(i)
    for i in cursor3:
        col3.append(i)
    cursor1.close()
    cursor2.close()
    cursor3.close()   ## 서버 닫기
    reference=np.array(col2[0][3:])  # 기준데이터
    userInfo=np.array(col3[0][:]) # 최근 일평균 영양데이터 
    percent = userInfo/reference #권장 섭취량과 사용유저 섭취량에 비율

    idPercentUsa = {}
    for i in range(len(col1)):
        idPercentUsa[col1[i][0]] =cos_sim(np.array(col1[i][1:]/reference),percent)    ## 사용유저와 전체 사용자간의 유사도 리스트 인덱스는 전체유저의 아이디
       
    
    recommend_list = sorted(idPercentUsa.items(), key = lambda item: item[1], reverse = True)  ## 유사도 높은 순으로 정렬
    usaUser =[]
    for i in range(len(col1)):
        usaUser.append(recommend_list[i][0])
    return usaUser

In [21]:
recommendUsaUser('ttt')


['ttt',
 'wo',
 'tnwl',
 'kimchman',
 'exp',
 '엄태균',
 '고준영죽빵',
 'asas',
 'zz',
 'ikujo',
 '1',
 '2',
 'ikujo123',
 'admin',
 'zxc',
 'yerim',
 '12344']

In [22]:
def recommendUsaNutri(userId) :  # 사용자가 가장 최근의 선택한 영양제 리스트를 호출하는 함수
    LOCATION = "C:\instantclient_21_6"
    os.environ["PATH"] = LOCATION
    connect = cx_Oracle.connect("campus_f_0707_2","smhrd2","project-db-stu.ddns.net:1524/xe")    ## DB접속
    cursor1 = connect.cursor() 
    cursor1.execute(f"select s_nutnm from S_NUTRIENTS where SS_USERID='{userId}' order by s_date desc")#유사도 높은 유저가 선택한 최신순 영양제 데이터
    # 아이디가 a인 사람인 섭취한 음식 일평균 영양 데이터
    col1=[]
    for i in cursor1:
        col1.append(i)   
    cursor1.close()   ## 서버 닫기
    recommendNutrients=[]
   
    for i in range(len(col1)):
        recommendNutrients.append(col1[i][0])  #영양제 이름란 리스트에
    recommendNutrientsOlRm = []
    for v in recommendNutrients:    ## 중복 제거
        if v not in recommendNutrientsOlRm:
            recommendNutrientsOlRm.append(v)
      
    return recommendNutrientsOlRm

In [23]:
def nutriSet3(userId) :  ## 추천하는 영양제를 3개로 마쳐주는 메소드
    recoNutirSet3=[]
    for i in recommendUsaNutri(recommendUsaUser(userId)[1]):
        recoNutirSet3.append(i)

    if len(recoNutirSet3)<3 :
        for j in  range(1,len(recommendUsaUser(userId))):
            if len(recoNutirSet3)==3 :
                    break
            for i in recommendUsaNutri(recommendUsaUser(userId)[j]):
                recoNutirSet3.append(i)
                if len(recoNutirSet3)==3 :
                    break
                
    return recoNutirSet3

In [24]:
nutriSet3('zxc')

['21st Century, 마그네슘, 250mg, 110정',
 'Natural Factors, 마그네슘시트레이트, 150mg, 캡슐 90정',
 '21st Century, 마그네슘, 250mg, 110정']

## jsp에서 id와 category를 받아 추천메뉴를 jsp에 전송
## jsp에서 id를 받아 추천영양제를 jsp에 전송

In [28]:
from urllib import parse
from unicodedata import category
from flask import Flask,request,redirect,render_template
from sympy import re

app = Flask(__name__) # flask 서버객체 생성
 
# route(경로): 외부에서 접근할 때 페이지 구분을 위한 경로지정
@app.route('/RecommendFood')
def food():
    id = request.args.get('id')
    category = request.args.get('category')
    ip = request.args.get('ip')
    if errorCheck(id) == 1 :
        menu=''
        rF=recommendFood(id,category)
        for i in rF :
            menu=menu+('menu='+i+'&')      ## 리스트를 쿼리스트링으로 보내기
        nutrients=''
        nutrientsReCo=''
        dN=deficientNutrient(id)
        rN=recommendNutrients(dN)
        rN2=nutriSet3(id)
        for i in rN :
            nutrients=nutrients+('nutrients='+i+'&')      ## 리스트를 쿼리스트링으로 보내기
        for i in rN2 :
            nutrientsReCo=nutrientsReCo+('nutrientsReCo='+i+'&')   
        url =f"http://{ip}:8082/ikujo/RecommendFood.jsp?category={category}&{nutrients}{menu}{nutrientsReCo}"
    else :
        url =f"http://{ip}:8082/ikujo/SelectFood.jsp?error=on"
    return redirect(url)

# main함수의 시작을 의미 
# 비유하자면 자바의 main(String[] args)메소드와 같은 역할
if __name__ == '__main__':
    # 웹서버주소(host)와 포트번호(port)를 지정하여 웹 서버 실행
    app.run(host='59.0.147.198', port='5000') 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://59.0.147.198:5000/ (Press CTRL+C to quit)


## 유사한 사용자 골라준느 플라스크 서버 (채팅기능을 위해)

In [193]:
from urllib import parse
from unicodedata import category
from flask import Flask,request,redirect,render_template
from sympy import re


app = Flask(__name__) # flask 서버객체 생성

@app.route('/')
def member():
    id = request.args.get('id')
    ip = request.args.get('ip')
    usaUserList=recommendUsaUser(id)
    usaUser=''
    for i in usaUserList :
        usaUser=usaUser+('usaUser='+i+'&') 
    
    url =f"http://{ip}:8082/ikujo/Chatting.jsp?{usaUser}"
    return redirect(url)    
# main함수의 시작을 의미
# 비유하자면 자바의 main(String[] args)메소드와 같은 역할
if __name__ == '__main__':
    # 웹서버주소(host)와 포트번호(port)를 지정하여 웹 서버 실행
    app.run(host='61.80.80.212', port='5001')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://59.0.147.198:5000/ (Press CTRL+C to quit)
59.0.147.198 - - [04/Aug/2022 11:06:40] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:13:06] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:14:06] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:15:29] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:17:24] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:18:37] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:20:11] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:25:18] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:25:46] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:26:17] "GET /?ip=59.0.147.198&id=zxc HTTP/1.1" 302 -
59.0.147.198 - - [04/Aug/2022 11:27:39] "GET /?ip=59.0.147.198&id=zxc HTTP/1.